In [ ]:
path = []


In [ ]:
path = [[0, 10], [1, 9], [2, 9], [3, 10], [4, 9], [5, 10], [6, 9], [7, 9], [8, 10], [9, 10], [10, 11], [11, 10], [12, 10], [13, 11],[14, 11], [15, 11], [16, 10], [17, 10], [18, 9], [19, 10]]

In [3]:
import cv2
import numpy as np
blank = cv2.imread("blank.png",0)
blank = cv2.resize(blank,(20,20))
# path = np.array(path)
# epsilon = 0.1*cv2.arcLength(path, True)
# path = cv2.approxPolyDP(path, epsilon, False)
# print(path)
pnts = []
for i in range(1,16):
    pts = [np.mean(path[0:15*(i)][0]),np.mean(path[0:15*(i)][1])]
    pnts.append(pts)
#     cv2.line(blank,(path[i][1],path[i][0]),(path[i+1][1],path[i+1][0]),0)
#     pnts = np.mean(path[i:i+15][0])
print(pnts)
blank = cv2.resize(blank,(680,480))
cv2.imshow("blank",blank)
q = cv2.waitKey(1)
if q == 27:
    cv2.destroyAllWindows()

NameError: name 'path' is not defined

In [3]:
import math
slp = []
for pix in pnts:
    slp.append(math.atan2(340-pix[1],pix[0]-240) - 1.57)

NameError: name 'pnts' is not defined

In [4]:
import math
print(slp)

[]


In [6]:
#output:: 6 category  
#input:: 15 slope 
#mode = xgboost
minibatch = []

In [1]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
import numpy as np
import time
import threading

In [2]:
import random
from collections import deque
class DQNAgent:
    def __init__(self):
        self.state_size = 15
        self.action_size = 6
        self.memory = deque(maxlen=pow(10,6))
        self.gamma = 0.8
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.99
        self.bst = xgb.Booster({'nthread':4})
        self.name = 'steerDeepQL'
        self.modelname = 'xgbmodel'
        self.load_bst()
    
    def xgbmodel_bld(self,data,label):
        param = {'base_score' : 1, 'max_depth': 7, 'eta': 0.1, #'updater':'refresh',
            #'process_type': 'update',
            'refresh_leaf': True,
            'reshape': True,
            'reg_alpha': 3, 
            'silent': 1, 
            'objective': 'multi:softprob',     
            'num_class': 6}  
        param['nthread'] = 4
        param['eval_metric'] = 'merror'
        num_round = 500  
        dtrain = xgb.DMatrix(data, label=label)
        evallist = [(dtrain, 'train'), (dtrain, 'eval')]
        self.bst = xgb.train(param, dtrain, num_round, evallist, early_stopping_rounds=3, xgb_model = self.name)
        self.save_bst()

        
    def load_bst(self):
        self.bst.load_model(self.name)
    
    def save_bst(self):
        self.bst.save_model(self.name)
        
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.bst.predict(xgb.DMatrix(state))
#         print('act_values',act_values)
        return np.argmax(act_values[0])  
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self):
        minibatch = self.memory
#         print(len(minibatch))
        bst_target = xgb.Booster({'nthread':4})
        bst_target.load_model(self.name)
        st = []
        tg = []
        for state, action, reward, next_state, done in minibatch:

            target = bst_target.predict(xgb.DMatrix(state))##target is Q matrix
            if done:
                target[0][action] = reward
            else:
                t = self.bst.predict(xgb.DMatrix(next_state))[0]
                target[0][action] = (1-self.gamma)*reward + self.gamma * np.amax(t)
            st.append(state)
            tg.append(np.argmax(target[0]))
        self.memory.clear()
        st = np.array(st)
        st = np.reshape(st, [-1, 15])
        tg = np.array(tg)
        tg = np.reshape(tg, [-1,1])
        print(st.shape, tg.shape)
        self.xgbmodel_bld(st, tg)

        
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    def decl_rew(self,state):
        rew = (np.sum((np.array([1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75]) - abs(state))*np.array([10,10,10,9,9,9,8,8,7,6,5,4,3,2,1])))/175
#         print('reward',rew)
#         print('state',state)
        done = False
        if rew > 0.74:
            done = True
        return rew,done
    
    def get_nextstate(self):
        ########
        stt = np.random.randint(175, size=15)/100
        return stt
    
def steer(GameOn):
    
    state_size = 15
    action_size = 6
    agent = DQNAgent()
    batch_size = 50
    state = np.zeros((15,), dtype=float)
    state = agent.get_nextstate()
#     agent.load_bst()
    count = 0
    bst_target = xgb.Booster({'nthread':4})
    st = []
    tg = []
    bst_target.load_model(agent.name)
    
    while GameOn:
        start = time.time()
        count += 1
        state = np.reshape(state, [1, state_size])
        action = agent.act(state)
#         execute_action(action)
        reward, done = agent.decl_rew(state)
#         reward = reward 
        next_state = agent.get_nextstate()
        if(next_state == []):
            continue;
        next_state = np.reshape(next_state, [1, state_size])
        
        agent.remember(state, action, reward, next_state, done)
#         target = bst_target.predict(xgb.DMatrix(state))
#         if done:
#             target[0][action] = reward
#         else:
#             t = agent.bst.predict(xgb.DMatrix(next_state))[0]
#             target[0][action] = (1-agent.gamma)*reward + agent.gamma * np.amax(t)
#         st.append(state)
#         tg.append(np.argmax(target[0]))
        
#         if count % 250 == 0:
#             count = 0
#             st = np.array(st)
#             st = np.reshape(st, [-1, 15])
#             tg = np.array(tg)
#             tg = np.reshape(tg, [-1,1])
#             agent.xgbmodel_bld(st, tg)
#             if agent.epsilon > agent.epsilon_min:
#                 agent.epsilon *= agent.epsilon_decay
#             st = []
#             tg = []
#             bst_target = None
#             bst_target = xgb.Booster({'nthread':4})
#             bst_target.load_model(agent.name)
#             print("success")
        state = next_state
        if done and len(agent.memory) >= 50 or len(agent.memory) >= 500 :
            print("replaying at {} xgboost".format(len(agent.memory)))
            t1 = threading.Thread(target=agent.replay, args=())
            t1.start()
#         time.sleep(0.03)
        print("FPS: ", 1.0 / (time.time() - start))
        

In [3]:
steer(True)

/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:118: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


FPS:  328.93922045329776
FPS:  8924.051063829787
FPS:  13706.875816993464
FPS:  9425.402247191012
FPS:  9258.949227373068
FPS:  11491.243835616438
FPS:  13148.288401253918
FPS:  13315.250793650794
FPS:  15141.891696750903
FPS:  15592.208178438661
FPS:  16131.938461538462
FPS:  15592.208178438661
FPS:  15827.562264150944
FPS:  15887.515151515152
FPS:  15363.750915750916
FPS:  16320.249027237354
FPS:  15887.515151515152
FPS:  16070.130268199233
FPS:  15650.388059701492
FPS:  15141.891696750903
FPS:  15363.750915750916
FPS:  16644.06349206349
FPS:  16384.0
FPS:  16070.130268199233
FPS:  16710.37450199203
FPS:  15827.562264150944
FPS:  16644.06349206349
FPS:  15196.753623188406
FPS:  8756.375782881001
FPS:  13486.508038585209
FPS:  16448.250980392157
FPS:  16448.250980392157
FPS:  16384.0
FPS:  16320.249027237354
FPS:  16448.250980392157
FPS:  16710.37450199203
FPS:  16644.06349206349
FPS:  16980.987854251012
FPS:  16912.516129032258
FPS:  16644.06349206349
FPS:  16710.37450199203
FPS:  16

FPS:  8422.29718875502
FPS:  7810.621973929236
FPS:  5652.700808625337
FPS:  7096.961082910321
FPS:  6502.796899224806
FPS:  8701.8755186722
FPS:  9510.893424036281
FPS:  10034.22009569378
FPS:  10407.70223325062
FPS:  10538.452261306533
FPS:  10205.119221411192
FPS:  10894.296103896104
FPS:  10645.441624365481
FPS:  10205.119221411192
FPS:  9822.72599531616
FPS:  10538.452261306533
FPS:  10538.452261306533
FPS:  9446.63063063063
FPS:  10645.441624365481
FPS:  10754.625641025641
FPS:  9446.63063063063
FPS:  10754.625641025641
FPS:  10407.70223325062
FPS:  10010.272076372315
FPS:  9198.035087719298
FPS:  6355.006060606061
FPS:  8473.341414141414
FPS:  8701.8755186722
FPS:  9000.652360515021
FPS:  9177.908096280087
FPS:  6990.506666666667
FPS:  7989.150476190476
FPS:  10433.592039800995
FPS:  8473.341414141414
FPS:  6326.250377073907
FPS:  7681.875457875458
FPS:  9619.963302752294
FPS:  10205.119221411192
FPS:  6626.072669826224
FPS:  6069.9044862518085
FPS:  11781.752808988764
FPS:  112

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-bf33f4a4271d>", line 58, in replay
    for state, action, reward, next_state, done in minibatch:
RuntimeError: deque mutated during iteration



 50.29443012170994
replaying at 504 xgboost
FPS:  109.2408907409819
replaying at 505 xgboost
FPS:  31.34522083551304
replaying at 506 xgboost
FPS:  26.571622246577427
replaying at 507 xgboost
FPS:  87.56741408827091
replaying at 508 xgboost
FPS:  35.643118759294666
replaying at 509 xgboost
FPS:  229.72417570380108
replaying at 510 xgboost
FPS:  49.92565259311281
replaying at 511 xgboost
FPS:  73.4747131470614
replaying at 512 xgboost
FPS:  65.2830282654713
replaying at 513 xgboost
FPS:  2003.0105062082139
replaying at 514 xgboost
FPS:  785.5973028657052
replaying at 515 xgboost
FPS:  79.65632893362454
replaying at 516 xgboost
FPS:  191.82730391035903
replaying at 517 xgboost
FPS:  12.195580367527333
replaying at 518 xgboost
FPS:  87.4104701567189
replaying at 519 xgboost
FPS:  186.92027273942688
replaying at 520 xgboost
FPS:  578.365140650855
replaying at 521 xgboost
FPS:  126.1862270224736
replaying at 522 xgboost
FPS:  31.430485511101786
replaying at 523 xgboost
FPS:  253.41695365838

...


KeyboardInterrupt

Exception in thread Thread-103:
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-bf33f4a4271d>", line 58, in replay
    for state, action, reward, next_state, done in minibatch:
RuntimeError: deque mutated during iteration

Exception in thread Thread-161:
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-bf33f4a4271d>", line 58, in replay
    for state, action, reward, next_state, done in minibatch:
RuntimeError: deque mutated during iteration

Exception 

Exception in thread Thread-109:
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-bf33f4a4271d>", line 58, in replay
    for state, action, reward, next_state, done in minibatch:
RuntimeError: deque mutated during iteration
Exception in thread Thread-155:
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-bf33f4a4271d>", line 58, in replay
    for state, action, reward, next_state, done in minibatch:
RuntimeError: deque mutated during iteration
Exception in thread Thread-141:


Exception in thread Thread-148:
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-bf33f4a4271d>", line 58, in replay
    for state, action, reward, next_state, done in minibatch:
RuntimeError: deque mutated during iteration
Exception in thread Thread-133:
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-bf33f4a4271d>", line 58, in replay
    for state, action, reward, next_state, done in minibatch:
RuntimeError: deque mutated during iteration
Exception in thread Thread-138:


Exception in thread Thread-115:
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-bf33f4a4271d>", line 58, in replay
    for state, action, reward, next_state, done in minibatch:
RuntimeError: deque mutated during iteration
Exception in thread Thread-104:
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-bf33f4a4271d>", line 58, in replay
    for state, action, reward, next_state, done in minibatch:
RuntimeError: deque mutated during iteration
Exception in thread Thread-128:


Exception in thread Thread-199:
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-bf33f4a4271d>", line 58, in replay
    for state, action, reward, next_state, done in minibatch:
RuntimeError: deque mutated during iteration

Exception in thread Thread-187:
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-bf33f4a4271d>", line 58, in replay
    for state, action, reward, next_state, done in minibatch:
RuntimeError: deque mutated during iteration

Exception in thread Thread-211

In [59]:
data = np.random.rand(100,15) # 5 entities, each contains 10 features
label = np.random.randint(6, size=100)
dtest = xgb.DMatrix([data[0]])
print(label.shape)
DQNAgent().xgbmodel_bld(data,label)
DQNAgent().bst.predict(dtest)
# np.argmax([DQNAgent().bst.predict(dtest),10])

(100,)
[0]	train-merror:0.41	eval-merror:0.41
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 10 rounds.
[1]	train-merror:0.33	eval-merror:0.33
[2]	train-merror:0.25	eval-merror:0.25
[3]	train-merror:0.25	eval-merror:0.25
[4]	train-merror:0.26	eval-merror:0.26
[5]	train-merror:0.25	eval-merror:0.25
[6]	train-merror:0.19	eval-merror:0.19
[7]	train-merror:0.19	eval-merror:0.19
[8]	train-merror:0.18	eval-merror:0.18
[9]	train-merror:0.16	eval-merror:0.16
[10]	train-merror:0.17	eval-merror:0.17
[11]	train-merror:0.15	eval-merror:0.15
[12]	train-merror:0.15	eval-merror:0.15
[13]	train-merror:0.13	eval-merror:0.13
[14]	train-merror:0.13	eval-merror:0.13
[15]	train-merror:0.13	eval-merror:0.13
[16]	train-merror:0.13	eval-merror:0.13
[17]	train-merror:0.12	eval-merror:0.12
[18]	train-merror:0.13	eval-merror:0.13
[19]	train-merror:0.12	eval-merror:0.12
[20]	train-merror:0.13	eval-merror:0.13
[21]	train-merro

array([ 0.5], dtype=float32)